<a href="https://colab.research.google.com/github/AkiraNom/data_analysis_notebook/blob/main/EDA_Titanic_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic EDA

dataset from [kaggle titanic data](https://www.kaggle.com/competitions/titanic/data)

In [ ]:
import pandas as pd
import numpy as np
